# Wrap-up Challenge

We've provided a file called `starwars.csv` in this folder (you should see it in the file explorer on the left). This is a *tidy* dataset that you're going to practice your Polars skills on by answering the following questions.

*Note: if you find it helpful to see similar commands on this same data using `dplyr` in R, they're available [here](https://psych252.github.io/psych252book/data-wrangling-1.html#looking-at-data)*

For clarity we're going to re-import everything you normally might when working with real data.

In [1]:
import polars as pl
from polars import col, when, lit
import polars.selectors as cs

## Inspecting data

Load the file and print the first few rows:

In [ ]:
# Your code here

In [ ]:
# Solution
sw = pl.read_csv('starwars.csv')
sw.head()

How many rows are in the DataFrame total?

In [ ]:
# Solution
sw.shape

How many missing values are in each column?

*Hint: checkout `.null_count()`*

In [ ]:
# Solution
sw.null_count()

How many unique characters are there?

*Hint: for quick operations you can directly refer to a rows/columns using slicing `[]` syntax*

In [ ]:
# Solution
sw['name'].n_unique()

87

Return a list of the unique "species" in the data and sort them alphabetically

In [ ]:
# Solution
sw['species'].unique().sort()

Return the number of observations for each unique "eye-color"

*Hint: checkout `.value_counts()`*

In [ ]:
# Solution
sw['eye_color'].value_counts()

## Wrangling Data

Ok lets use the **contexts** and **expressions** we've learned about filter and aggregate this data in different ways.

Filter rows to get character's whose "gender" is "masculine" and "height" is above the median height of all characters.

How many rows in the result?

In [ ]:
# Solution
sw.filter(
    col('gender').eq('masculine') & col('height').ge(col('height').median())
)

Expand your filter to also exclude rows where a character's "mass" is missing (null).

How many rows remain?

In [ ]:
# Your code here


In [ ]:
# Solution
sw.filter(
    col('gender').eq('masculine') & col('height').ge(col('height').median()) & col('mass').is_not_null()
)

Expand your filter to include the previous result *or* characters whose "skin_color" is 'dark' or 'pale'

How many rows in this result?

In [ ]:
# Your code here


In [ ]:
# Solution
sw.filter(
    (col('gender').eq('masculine') & 
     col('height').ge(col('height').median()) & 
     col('mass').is_not_null()) |  
     col('skin_color').is_in(['dark', 'pale'])
)

What is the average height and average mass of characters who are from a "homeworld" whose name starts with the letters 'Co'?


*Hint: you can "chain" a `.select()` onto the result of a `.filter()`*

In [ ]:
# Your code


In [ ]:
sw.filter(
    col('homeworld').str.starts_with('Co')
).select(
    col('height','mass').mean()
)

Summarize the mean and standard-deviation of the "height" and "mass" of all non-Human characters, split by "sex", and sort the result by the mean height in *descending order*.


*Hint: you can chain a `.group_by()` after a `.filter()`, and chain `.sort()` at the end*

In [ ]:
# Your code here


In [ ]:
# Solution
sw.filter(
    ~col('species').eq('Human')
).group_by('sex').agg(
    height_mean = col('height').mean(),
    height_std = col('height').std(),
    mass_mean = col('mass').mean(),
    mass_std = col('mass').std(),
).sort('height_mean', descending=True)

Extend your summary to filter out any values of "sex" that are "hermaphroditic" or "none" (not missing/null!)

In [ ]:
# Your code here


In [ ]:
# Solution
sw.filter(
    ~col('species').eq('Human') & ~col('sex').is_in(['hermaphroditic', 'none'])
).group_by('sex').agg(
    height_mean = col('height').mean(),
    height_std = col('height').std(),
    mass_mean = col('mass').mean(),
    mass_std = col('mass').std(),
).sort('height_mean', descending=True)

Add a column to the original DataFrame called "mass_lbs" that converts "mass" to lbs by multiplying by 2.2 and drop all remaining columns except: name, height, and species.

Remove any rows with null values and save the result to a new DataFrame called `sw_processed`

In [ ]:
# Your code here


In [ ]:
# Solution
sw_processed = (
    sw.with_columns(
        mass_lbs = col('mass') * 2.2
    ).select(['name','height','species','mass_lbs']).drop_nulls()
)
sw_processed

Use the new DataFrame you created `sw_processed` to add 2 additional columns called "height_z_species" and "mass_lbs_z_species" that are z-scored versions of height and mass_lbs by "species"

Save the DataFrame by overwriting the previous variable `sw_processed`

In [ ]:
# Your code here


In [ ]:
# Solution
zscore = lambda name: ( col(name) - col(name).mean() ) / col(name).std()

sw_processed = sw_processed.with_columns(
    height_z_species = zscore('height').over('species'),
    mass_lbs_z_species = zscore('mass_lbs').over('species'),
)

sw_processed

Whats the shape of the result?

In [ ]:
# Solution
sw_processed.shape

(56, 6)

How many unique characters are in the result?

In [ ]:
sw_processed['name'].n_unique()

56

Drop all non-zscored numeric columns and once again save the result to `sw_processed`, overwriting your previous DataFrame

In [ ]:
# Your code here


In [ ]:
# Solution
sw_processed = sw_processed.drop(['height', 'mass_lbs'])
sw_processed

name,species,height_z_species,mass_lbs_z_species
str,str,f64,f64
"""Luke Skywalker""","""Human""",-0.71983,-0.222945
"""C-3PO""","""Droid""",0.519167,0.102877
"""R2-D2""","""Droid""",-0.84605,-0.739734
"""Darth Vader""","""Human""",1.897735,2.828964
"""Leia Organa""","""Human""",-2.639378,-1.671308
…,…,…,…
"""Shaak Ti""","""Togruta""",null,null
"""Grievous""","""Kaleesh""",null,null
"""Tarfful""","""Wookiee""",0.707107,0.707107


What is Leia Organa's height in z-scores?

Who's heavier in z-scores, Owen Lars or Darth Vader and by how much approximately?

In [ ]:
# Your code here


In [ ]:
# Solution
sw_processed.filter(
    col('name').is_in(['Owen Lars', 'Darth Vader'])
).select('name', 'mass_lbs_z_species')

Add in a column to your DataFrame `sw_processed` called "height_category" that splits characters based upon the z-scored height. 

If their height is between (or including) -2 and 2 their height_category should be in the "normal".  
If their height is < -2 their height_category should be in the "short".  
If their height is > 2 their height_category should be in the "tall".  

Take the resulting column along with the "name" and "species" columns and save them to a new DataFrame variable called `sw_heights`

In [ ]:
# Your code here


In [ ]:
# Solution
sw_heights = sw_processed.with_columns(
    height_category = when(col('height_z_species').ge(-2).and_(col('height_z_species').le(2)))
                      .then(lit('normal'))
                      .when(col('height_z_species').lt(-2))
                      .then(lit('short'))
                      .otherwise(lit('tall'))
                      
).select('name','species', 'height_category')
sw_heights

Summarize the number of unique species by "height_category". How many of each are there?

In [ ]:
# Your code here


In [ ]:
# Solution
sw_heights.group_by('height_category').agg(
    col('species').n_unique()
)

height_category,species
str,u32
"""normal""",5
"""tall""",26
"""short""",1


Are there any humans in the "tall" category?

*Hint: try using `.unique()` inside of a `.group_by()` context. Then use a `.select()` context on the result that includes an expression using the [`.list` attribute operations](https://docs.pola.rs/api/python/stable/reference/expressions/list.html) to check if 'Human' is one of the list items*

In [ ]:
# Your code here


In [ ]:
# Solution
sw_heights.group_by('height_category').agg(
    col('species').unique()
).select(
    col('height_category'),
    col('species').list.contains('Human')
)